# Validate WILDS Metadata Extension

In [30]:
import json
import pathlib
import os

import pandas as pd
from wilds import get_dataset
import geemap
import ee

file_path = os.path.abspath('')

PROJECT_ROOT = pathlib.Path(file_path).parent.parent.resolve()
DATA_DIR = PROJECT_ROOT / "data"

if not (os.path.exists(PROJECT_ROOT) and os.path.exists(DATA_DIR)):
    raise NotADirectoryError()

EE_PROJECT_NAME = 'seeing-the-big-picture'

try:
    ee.Authenticate()
    ee.Initialize(project=EE_PROJECT_NAME)
except Exception as e:
    print("Please authenticate Earth Engine: earthengine authenticate")
    raise

In [31]:
dataset = get_dataset(dataset="fmow")
metadata = pd.read_csv(DATA_DIR / "rgb_metadata_wilds_extended.csv")

In [52]:
sample_meta = metadata.sample()
sample_idx = sample_meta.index[0]
img_pil, y_tensor, _ = dataset[sample_idx]  # Triple: (PIL, label, metadata)
img_center_lon, img_center_lat, img_span_deg = sample_meta[
    "img_center_lon"].values[0], sample_meta["img_center_lat"].values[0], sample_meta["img_span_deg"].values[0]
bbox = [img_center_lon - img_span_deg, img_center_lat - img_span_deg,
        img_center_lon + img_span_deg, img_center_lat + img_span_deg]

In [ ]:
# Create GEE map
def scale_l8(image):
    return (image
            .select(['SR_B2', 'SR_B3', 'SR_B4'])
            .multiply(0.0000275)
            .add(-0.2))


m = geemap.Map(center=[img_center_lat, img_center_lon], zoom=17)
region = ee.Geometry.Rectangle(bbox)

# Load satellite image from

l8 = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    .filterBounds(region)
    .map(scale_l8)
)

least_cloudy = l8.sort('CLOUD_COVER').first()

rgb_mask = (least_cloudy
            .select(['SR_B2', 'SR_B3', 'SR_B4'])
            .mask()
            .reduce(ee.Reducer.min()))

coverage_dict = rgb_mask.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=region,
    scale=30,
    maxPixels=1e7
)

least_cloudy_coverage = ee.Number(coverage_dict.get('min'))
rgb_ok = least_cloudy_coverage.gte(0.99)

composite = l8.median()
context = ee.Image(ee.Algorithms.If(rgb_ok, least_cloudy, composite))

m.addLayer(
    context,
    {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.3},
    'Landsat8'
)

# TODO: Second image
min_lon2, min_lat2, max_lon2, max_lat2 = bbox_2

# Save PIL image to a temporary file (PNG or JPG)
tmp_path = "tmp_overlay.png"
img_pil.save(tmp_path)

# Create an ImageOverlay with geographic bounds
overlay = geemap.ImageOverlay(
    url=tmp_path,
    bounds=((min_lat2, min_lon2), (max_lat2, max_lon2)),
    name="PIL overlay"
)

# Add overlay to the same map
m.add_layer(overlay)

m

Map(center=[np.float64(32.8275151828724), np.float64(-96.68286292187284)], controls=(WidgetControl(options=['p…